In [2]:
import tensorflow as tf
import pandas as pd
import os
import re
import numpy as np
!pip install --user keras-bert

  Stored in directory: /home/purva/.cache/pip/wheels/09/60/51/9e65b52719417b7517c2dba2737be82802164f7699545abd10
  Stored in directory: /home/purva/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Stored in directory: /home/purva/.cache/pip/wheels/ec/9f/95/e2bf2acfd101fd5f252f4d399660c810b7367ccd4412d6c585
  Stored in directory: /home/purva/.cache/pip/wheels/95/49/02/4eda210bc4c37ff1d45311665bceb790881dbea92b27b025a5
  Stored in directory: /home/purva/.cache/pip/wheels/c3/9b/9e/f4072915f660e90bb3638332276f4de80476f3afcb5d010d6f
  Stored in directory: /home/purva/.cache/pip/wheels/3a/58/cc/007f963a01c8304c793f9e887d50ff6e7c495343287c01a6a2
  Stored in directory: /home/purva/.cache/pip/wheels/c6/53/9b/41d5808349a3ba2f1fd6ad07ecb91141e0bf8cce66e9f6bcf8
  Stored in directory: /home/purva/.cache/pip/wheels/cc/dc/17/84258b27a04cd38ac91998abe148203720ca696186635db694
Successfully built keras-bert keras-pos-embd keras-transformer keras-multi-head keras-layer-norm

In [5]:
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K

In [7]:
sess=tf.Session()

In [8]:
# Params for bert model and tokenization
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
max_seq_length = 256

In [11]:
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  

In [18]:
from tensorflow.keras import utils
file="aclImdb.tar.gz"
origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset=tf.keras.utils.get_file(file,origin,extract="True")


In [19]:
dataset

'/home/purva/.keras/datasets/aclImdb.tar.gz'

In [20]:
train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))

In [21]:
train_df.head()

,sentence,sentiment,polarity
0,"Lots of singing and dancing in this one, espec...",7,1
1,This is a film that belongs firmly to the 50's...,2,0
2,"America. A land of freedom, of hope and of dre...",1,0
3,"A very silly movie, this starts with a soft po...",7,1
4,Impenetrable rubbish. This has to be one of th...,1,0


In [23]:
test_df.head()

,sentence,sentiment,polarity
0,My father has worked with top secret informati...,7,1
1,I knew I was in for a LONG 90 minutes when the...,1,0
2,Milos Forman's original HAIR was the perfect m...,10,1
3,The story of Sweeney Todd evokes memories of t...,10,1
4,"""Best in Show"" is certainly Christopher Guest'...",9,1


In [24]:
train_text = train_df['sentence'].tolist()

In [25]:
train_text

['Lots of singing and dancing in this one, especially by Gene Kelly. Two sailors go on liberty to see if they can find love and romance. They meet up with a woman who is trying to break into show business. Musical lovers only.',
 'This is a film that belongs firmly to the 50\'s. Very surprising that American Film Institute has chosen this one for one for the best 100 American movies of all-time. I have seen practically all of the movies on that list, and this one is by far the most disappointing one of those. Musical numbers (and there many, many of them) are VERY overlong and boring, and have absolute no connection with the story. The end of the movie has horribly over-long ballet sequency, which naturally has no real relation to the story of the movie. It must be admitted, that it is very well made, the music is OK, and the dancing done with the highest professional standard - but there is no real reason why the sequence is included in the movie.<br /><br />The main character of the 

In [32]:
# Create datasets (Only take up to max_seq_length words for memory)
train_text = train_df['sentence'].tolist()
train_text = [' '.join(t.split()[0:max_seq_length]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = train_df['polarity'].tolist()

test_text = test_df['sentence'].tolist()
test_text = [' '.join(t.split()[0:max_seq_length]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = test_df['polarity'].tolist()

!pip install --user hub-toolbox


In [33]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

def convert_single_example(tokenizer, example, max_seq_length=256):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        input_ids = [0] * max_seq_length
        input_mask = [0] * max_seq_length
        segment_ids = [0] * max_seq_length
        label = 0
        return input_ids, input_mask, segment_ids, label

    tokens_a = tokenizer.tokenize(example.text_a)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, example.label

def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels).reshape(-1, 1),
    )

def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

# Instantiate tokenizer
tokenizer = create_tokenizer_from_hub_module()

# Convert data to InputExample format
train_examples = convert_text_to_examples(train_text, train_label)
test_examples = convert_text_to_examples(test_text, test_label)

# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

NameError: name 'hub' is not defined

In [34]:
class BertLayer(tf.layers.Layer):
    def __init__(self, n_fine_tune_layers=10, **kwargs):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            bert_path,
            trainable=self.trainable,
            name="{}_module".format(self.name)
        )

        trainable_vars = self.bert.variables

        # Remove unused layers
        trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]

        # Select how many layers to fine tune
        trainable_vars = trainable_vars[-self.n_fine_tune_layers :]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)
            
        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
            "pooled_output"
        ]
        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [35]:
from keras_bert import Tokenizer

token_dict = {
    '[CLS]': 0,
    '[SEP]': 1,
    'un': 2,
    '##aff': 3,
    '##able': 4,
    '[UNK]': 5,
}
tokenizer = Tokenizer(token_dict)
print(tokenizer.tokenize('unaffable'))  # The result should be `['[CLS]', 'un', '##aff', '##able', '[SEP]']`
indices, segments = tokenizer.encode('unaffable')
print(indices)  # Should be `[0, 2, 3, 4, 1]`
print(segments)  # Should be `[0, 0, 0, 0, 0]`

print(tokenizer.tokenize(first='unaffable', second='钢'))
# The result should be `['[CLS]', 'un', '##aff', '##able', '[SEP]', '钢', '[SEP]']`
indices, segments = tokenizer.encode(first='unaffable', second='钢', max_len=10)
print(indices)  # Should be `[0, 2, 3, 4, 1, 5, 1, 0, 0, 0]`
print(segments)  # Should be `[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]`

['[CLS]', 'un', '##aff', '##able', '[SEP]']
[0, 2, 3, 4, 1]
[0, 0, 0, 0, 0]
['[CLS]', 'un', '##aff', '##able', '[SEP]', '钢', '[SEP]']
[0, 2, 3, 4, 1, 5, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]


Using TensorFlow backend.


In [37]:
import keras
from keras_bert import get_base_dict, get_model, gen_batch_inputs


# A toy input example
sentence_pairs = [
    [['all', 'work', 'and', 'no', 'play'], ['makes', 'jack', 'a', 'dull', 'boy']],
    [['from', 'the', 'day', 'forth'], ['my', 'arm', 'changed']],
    [['and', 'a', 'voice', 'echoed'], ['power', 'give', 'me', 'more', 'power']],
]


# Build token dictionary
token_dict = get_base_dict()  # A dict that contains some special tokens
for pairs in sentence_pairs:
    for token in pairs[0] + pairs[1]:
        if token not in token_dict:
            token_dict[token] = len(token_dict)
token_list = list(token_dict.keys())  # Used for selecting a random word


# Build & train the model
model = get_model(
    token_num=len(token_dict),
    head_num=5,
    transformer_num=12,
    embed_dim=25,
    feed_forward_dim=100,
    seq_len=20,
    pos_num=20,
    dropout_rate=0.05,
)
model.summary()

def _generator():
    while True:
        yield gen_batch_inputs(
            sentence_pairs,
            token_dict,
            token_list,
            seq_len=20,
            mask_rate=0.3,
            swap_sentence_rate=1.0,
        )

model.fit_generator(
    generator=_generator(),
    steps_per_epoch=1000,
    epochs=100,
    validation_data=_generator(),
    validation_steps=100,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    ],
)


# Use the trained model
inputs, output_layer = get_model(
    token_num=len(token_dict),
    head_num=5,
    transformer_num=12,
    embed_dim=25,
    feed_forward_dim=100,
    seq_len=20,
    pos_num=20,
    dropout_rate=0.05,
    training=False,      # The input layers and output layer will be returned if `training` is `False`
    trainable=False,     # Whether the model is trainable. The default value is the same with `training`
    output_layer_num=4,  # The number of layers whose outputs will be concatenated as a single output.
                         # Only available when `training` is `False`.
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 20)           0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 20, 25), (28 700         Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Em

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/100
1000/1000 [==============================] - 47s 47ms/step - loss: 3.9827 - MLM_loss: 3.3349 - NSP_loss: 0.6479 - val_loss: 3.8986 - val_MLM_loss: 3.3170 - val_NSP_loss: 0.5815
Epoch 2/100
1000/1000 [==============================] - 37s 37ms/step - loss: 3.7901 - MLM_loss: 3.2959 - NSP_loss: 0.4942 - val_loss: 3.5401 - val_MLM_loss: 3.2719 - val_NSP_loss: 0.2682
Epoch 3/100
1000/1000 [==============================] - 37s 37ms/step - loss: 3.4818 - MLM_loss: 3.2706 - NSP_loss: 0.2112 - val_loss: 3.2905 - val_MLM_loss: 3.2563 - val_NSP_loss: 0.0342
Epoch 4/100
1000/1000 [==============================] - 37s 37ms/step - loss: 3.3200 - MLM_loss: 3.2305 - NSP_loss: 0.0895 - val_loss: 3.2407 - val_MLM_loss: 3.2101 - val_NSP_loss: 0.0305
Epoch 5/100
1000/1000 [==============================] - 37s 37ms/step - loss: 3.2411 - MLM_loss: 3.1726 - NSP_loss: 0.0685 - val_loss: 3.1626 - val_MLM_loss: 3.1267 

1000/1000 [==============================] - 37s 37ms/step - loss: 0.0792 - MLM_loss: 0.0651 - NSP_loss: 0.0141 - val_loss: 0.0164 - val_MLM_loss: 0.0157 - val_NSP_loss: 6.8147e-04
Epoch 43/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.0785 - MLM_loss: 0.0648 - NSP_loss: 0.0136 - val_loss: 0.0171 - val_MLM_loss: 0.0162 - val_NSP_loss: 8.9998e-04
Epoch 44/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.0616 - MLM_loss: 0.0493 - NSP_loss: 0.0122 - val_loss: 0.0156 - val_MLM_loss: 0.0144 - val_NSP_loss: 0.0013
Epoch 45/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.0666 - MLM_loss: 0.0514 - NSP_loss: 0.0152 - val_loss: 0.0146 - val_MLM_loss: 0.0132 - val_NSP_loss: 0.0014
Epoch 46/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.0505 - MLM_loss: 0.0430 - NSP_loss: 0.0076 - val_loss: 0.0159 - val_MLM_loss: 0.0146 - val_NSP_loss: 0.0013
Epoch 47/100
1000/1000 [==============================]

In [38]:
import numpy as np
from keras_bert import AdamWarmup, calc_train_steps

train_x = np.random.standard_normal((1024, 100))

total_steps, warmup_steps = calc_train_steps(
    num_example=train_x.shape[0],
    batch_size=32,
    epochs=10,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

In [41]:
train_x

array([[-0.16591168,  0.0577722 , -0.67752595, ...,  0.19074454,
         0.47084408, -1.21765546],
       [-0.04327323, -0.72274694, -1.16527493, ..., -0.68091306,
         0.07671177, -0.48639343],
       [ 0.3232615 ,  1.94414692, -0.0870427 , ...,  0.76331891,
         1.49164916,  1.0275805 ],
       ...,
       [ 2.72206787, -1.35400665,  1.33842572, ..., -0.21871439,
         0.94443369,  1.26744446],
       [-1.42257809,  0.37405057,  0.97750875, ..., -0.71435838,
         0.30859914,  0.29654619],
       [ 0.23502121, -0.946481  , -0.54120445, ...,  0.21144215,
         0.20832145, -0.12266884]])